In [2]:
import numpy as np
from scipy.integrate import odeint
import matplotlib.pyplot as plt
import csv
import pandas as pd
import itertools
import math
import os
import sympy as sp
import random
from sklearn.model_selection import train_test_split
from sympy import symbols, Function, diff
output_directory = r'C:\Users\mebratie\Desktop\KR\KR_Regression\Data\Henon_non_chaotic_T10_20_25_30_35_40_45_50\Henon_non_chaotic_T10_20'
def generate_random_values():
    a1 = -0.5
    a2 = 0.5
    a3 = -0.1
    a4 = 0.1
    x1 = np.random.uniform(a1, a2)
    x2 = np.random.uniform(a1, a2)
    x3 = np.random.uniform(a3, a4)
    x4 = np.random.uniform(a3, a4)
    return x1, x2, x3, x4
def generate_random_values_based_on_c():
    a1 = -0.5
    a2 = 0.5
    a3 = -0.1
    a4 = 0.1
    x1 = np.random.uniform(a1, a2)
    x2 = np.random.uniform(a1, a2)
    x3 = np.random.uniform(a3, a4)
    x4 = np.random.uniform(a3, a4)
    return x1, x2, x3, x4
def generate_data(initial_conditions):
    def normalize(vector):
        norm = np.linalg.norm(vector)
        if norm == 0: 
            return vector
        return vector / norm
    def normalized_system(y, t):
        x1, x2, x3, x4 = y
        f = np.array([x3, x4, -x1 - 2*x1*x2, -x2 - x1**2 + x2**2])
        normalized_f = normalize(f)
        return normalized_f
    def compute_energy(x1, x2, x3, x4):
        return 0.5 * (x3**2 + x4**2) + 0.5 * (x1**2 + x2**2) + x1**2 * x2 - (1/3) * x2**3
    num_trajectories = 10
    t = np.linspace(0, 10, 20) # 40 data points per trajectory
    all_trajectory_data = []
    initial_conditions_to_print = []
    print("Initial data (x1, x2, x3, x4):")
    for i, initial_condition in enumerate(initial_conditions):
        print(f"({initial_condition[0]}, {initial_condition[1]}, {initial_condition[2]}, {initial_condition[3]})")
        sol = odeint(normalized_system, initial_condition, t)
        all_trajectory_data.append(sol)
        final_state = sol[-1, :]
        E = compute_energy(*final_state)
        initial_conditions_to_print.append((initial_condition, E))
    num_variables = 4 # Adjust number of variables that we need for the regression accordingly
    column_names = [f'x{i+1}' for i in range(num_variables)]
    column_names.append('trajectory')
    with open('50.csv', 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(column_names)
        for r, data in enumerate(all_trajectory_data):
            for j in range(len(t)):
                row = data[j].tolist() + [r + 1]
                writer.writerow(row) 
    for r, (initial_conditions, E) in enumerate(initial_conditions_to_print):
        print(f"Energy for Trajectory {r+1}: {E}")
    output_directory1 = r'C:\Users\mebratie\Desktop\KR\KR_Regression\results\Henon_non_chaotic_T10_20_25_30_35_40_45_50\Henon_non_chaotic_T10_20'
    plt.figure(figsize=(10, 6))
    for i, sol in enumerate(all_trajectory_data):
        for j in range(sol.shape[1]):
            plt.plot(t, sol[:, j])
    plt.savefig(os.path.join(output_directory1, 'trajectory.png'))
    plt.close()
def split_data():
    trajectories = {}
    column_names = None
    with open('C:\\Users\\mebratie\\Desktop\\KR\\KR_Regression\\Data\\Henon_non_chaotic_T10_20_25_30_35_40_45_50\\Henon_non_chaotic_T10_20\\trainingp_data50.csv', 'r') as trainfile:
        reader = csv.DictReader(trainfile)
        column_names = reader.fieldnames
        for row in reader:
            trajectory = float(row['trajectory'])
            if trajectory not in trajectories:
                trajectories[trajectory] = []
            trajectory_data = {key: float(value) for key, value in row.items()}
            trajectories[trajectory].append(trajectory_data)
    for traj_points in trajectories.values():
        random.shuffle(traj_points)
    num_points_per_file = len(next(iter(trajectories.values()))) // 5  # divide into five splits (n stratify)
    for i in range(5):  # Five-fold cross-validation
        output_filename = f'B50{i+1}.csv'
        with open(os.path.join(output_directory, output_filename), 'w', newline='') as output_file:
            writer = csv.DictWriter(output_file, fieldnames=column_names)
            writer.writeheader()
            for trajectory, points in trajectories.items():
                for point in points[i * num_points_per_file: (i + 1) * num_points_per_file]:
                    writer.writerow(point)
if __name__ == "__main__":
    x1, x2, x3, x4 = generate_random_values()
    initial_conditions = [generate_random_values_based_on_c() for _ in range(10)]  # number of trajectories
    generate_data(initial_conditions)
    data = np.genfromtxt('50.csv', delimiter=',', names=True)
    training_data = []
    holdout_data = []
    for r in range(1, 11):  # this represents the number of initial data is 5. i.e., (1,6) means 5 initial data
        trajectory_subset = data[data['trajectory'] == r]
        train_set, holdout_set = train_test_split(trajectory_subset, test_size=0.2, random_state=42)
        training_data.extend(train_set)
        holdout_data.extend(holdout_set)
    column_names = data.dtype.names
    with open(os.path.join(output_directory, 'trainingp_data50.csv'), 'w', newline='') as trainfile:
        writer = csv.writer(trainfile)
        writer.writerow(column_names)
        for row in training_data:
            writer.writerow([row[col] for col in column_names])
    with open(os.path.join(output_directory, 'holdoutp_data50.csv'), 'w', newline='') as holdfile:
        writer = csv.writer(holdfile)
        writer.writerow(column_names)
        for row in holdout_data:
            writer.writerow([row[col] for col in column_names])
    split_data()

Initial data (x1, x2, x3, x4):
(0.05848715310380492, 0.3258538995913034, 0.07801156687890182, 0.021140866064325664)
(-0.17266761954851917, 0.3558231354655911, -0.028071394837600377, -0.07433868276385919)
(-0.12618889773721442, 0.3228667575663995, -0.0455385748593153, 0.029010856614875624)
(-0.3376814329510164, 0.09991110874262299, -0.0961553814792044, -0.07270543028990675)
(-0.1768286692050023, -0.02772855634854865, -0.05524691988608979, 0.008251900074324456)
(-0.1523615853948972, 0.07629836685706237, -0.0547969645052274, 0.09882034444879437)
(-0.32365824277528554, 0.04375134596517827, 0.052385355317017546, 0.08822242988147647)
(0.06882175406325064, 0.15068799133325972, -0.06257451956752909, -0.09191206494933124)
(-0.030855764266584762, 0.3011691021630457, -0.0871359474846659, 0.0581260663585407)
(0.2738703501605855, 0.3281880645177948, 0.05392276722113029, 0.020547130653149143)
Energy for Trajectory 1: 0.04764862482537965
Energy for Trajectory 2: 0.07696088254463665
Energy for Traject